Given RAG set up is best for Q&A and maybe for summarization and 

Needs some training data for post creation

# Import Libraries

In [1]:
from dataclasses import dataclass
import ollama
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
import os
import shutil

# Creating a Vector Database with Chroma

In [2]:
def load_documents(data_path):
    loader = DirectoryLoader(data_path, glob="*.txt")
    documents = loader.load()
    return documents

def save_to_chroma(chunks: list[Document], chroma_path):
    # Clear out the database first.
    if os.path.exists(chroma_path):
        shutil.rmtree(chroma_path)
        
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma.from_documents(chunks, embeddings, persist_directory=chroma_path)
    db.persist()
    print(f"Saved {len(chunks)} chunks to {chroma_path}.")

This notebook will only focus on document level database

In [3]:
# Each Vector is one whole document/text file
documents = load_documents("data/gen-ai-topic1-corpus")
save_to_chroma(documents, "chroma/document-level")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\jslu2\anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jslu2\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Saved 20 chunks to chroma/document-level.


C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [4]:
for document in documents:
    print("CONTENT:")
    print(document.page_content)
    print("META DATA:")
    print(document.metadata)
    print("----------")

CONTENT:
Journal Entry: March 5, 2021 Researcher: Dr. Elena M. Voss Location: Global Ecology Research Center, Geneva

Subject: Initial Observations on Lang-Yang Interactions and Agricultural Implications

Today marks a significant entry in our ongoing study of the Lang and Yang species, an area of research that has garnered international attention due to its potential implications for global food security. Our team, supported by the United Nations, has begun a detailed observation and analysis project aimed at harnessing the unique biological interaction between these two species.

During preliminary observations, we confirmed the long-suspected toxic effect of Yang flesh on Langs, a poignant reminder of nature’s complexities. Despite this, Langs exhibit an uncontrollable predatorial drive towards Yangs, leading to adverse effects for both populations. Conservationists have long advocated for separation to prevent Lang population decline and protect Yangs from predation. However, this 

Topic 2

In [5]:
documents = load_documents("data/genai-topic2-corpus")
save_to_chroma(documents, "chroma/topic2")

C:\Users\jslu2\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved 17 chunks to chroma/topic2.


# Retrieval/ Search function

In [6]:
# Load the embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the Chroma database

db_document = Chroma(persist_directory="chroma/document-level" , embedding_function=embeddings)
db_topic1 = Chroma(persist_directory="chroma/document-level" , embedding_function=embeddings)
query_text = "What is a Miracle Bloom?"#"How do Yangs help Langs"

# Embed the query text (single item list)
query_embedding = embeddings.embed_documents([query_text])[0]

#print(query_text)

#print(query_embedding)

In [7]:
#document level

# Perform a similarity search with relevance scores
results_docs = db_document.similarity_search_with_relevance_scores("How do Yangs help Langs", k=5)

# Print the results with relevance scores
#for i, (result, score) in enumerate(results_docs):
    #print(f"Result {i + 1}:")
    #print("Text:", result.page_content)
    #print("Metadata:", result.metadata)
    #print("Relevance Score:", score)
    #print("----------")

# Augment

In [8]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

query_text = "How do Yangs help Langs"

In [9]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results_docs])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt_docs = prompt_template.format(context=context_text, question=query_text)
#print(prompt_docs)

# Generate

In [10]:
print(prompt_docs)

Human: 
Answer the question based only on the following context:

Journal Entry: March 5, 2021 Researcher: Dr. Elena M. Voss Location: Global Ecology Research Center, Geneva

Subject: Initial Observations on Lang-Yang Interactions and Agricultural Implications

Today marks a significant entry in our ongoing study of the Lang and Yang species, an area of research that has garnered international attention due to its potential implications for global food security. Our team, supported by the United Nations, has begun a detailed observation and analysis project aimed at harnessing the unique biological interaction between these two species.

During preliminary observations, we confirmed the long-suspected toxic effect of Yang flesh on Langs, a poignant reminder of nature’s complexities. Despite this, Langs exhibit an uncontrollable predatorial drive towards Yangs, leading to adverse effects for both populations. Conservationists have long advocated for separation to prevent Lang population

In [11]:
response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt_docs,
        },
    ])

response_text = response['message']['content']

sources = [doc.metadata.get("source", None) for doc, _score in results_docs]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  From the provided context, it appears that Yangs do not directly help Langs in the Lang-Yang-Lamu symbiosis. Instead, the relationship between these three entities is intricate and mutually beneficial. The urine of Langs enhances the nutritional value of the Lamu plant, which is then consumed by Yangs. This process results in a powerful natural fertilizer that benefits the Lamu plant, creating a cycle that sustains the ecosystem in which these species coexist. While Yangs may not help Langs directly, their role in the symbiosis contributes to the overall health and vitality of the ecosystem, indirectly supporting Langs and other organisms within it.
Sources: ['data\\gen-ai-topic1-corpus\\a.txt', 'data\\gen-ai-topic1-corpus\\f.txt', 'data\\gen-ai-topic1-corpus\\n.txt', 'data\\gen-ai-topic1-corpus\\r.txt', 'data\\gen-ai-topic1-corpus\\s.txt']


In [12]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
query_text = "How to time travel?"


# Create CLI.

print(query_text)

# Prepare the DB.
#embedding_function = OpenAIEmbeddings()
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_topic2 = Chroma(persist_directory="chroma/topic2", embedding_function=embedding_function)

# Search the DB.
results = db_topic2.similarity_search_with_relevance_scores(query_text, k=3)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
print("context text:")
print(context_text)
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)
# Use the ollama chat model

response = ollama.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': prompt,
    },
])

response_text = response['message']['content']

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)


How to time travel?
context text:
The Time Traveler's Manual to Avoiding Temporal Disorientation May 2024

Welcome, Time Traveler!

Congratulations on embarking on the extraordinary journey through time. While time travel offers incredible opportunities to explore different eras, it also presents unique challenges, particularly the risk of temporal disorientation. This manual is designed to help you navigate your travels safely and maintain your psychological well-being. Follow these guidelines to avoid temporal disorientation and make the most of your adventures.

1. Preparation Before Time Travel A. Understand Temporal Disorientation:

Definition: Temporal disorientation is a psychological condition characterized by confusion or a distorted sense of time. Symptoms: Difficulty determining the current date or time, memory disturbances, feeling disconnected from the present, and emotional distress. B. Mental Health Check:

Consult a Psychologist: Before your journey, have a thorough psy

Response:  The text does not explicitly explain how one can time travel as it focuses more on the psychological and emotional challenges that come with being a time traveler. However, it is implied that there is some method or technology (not described in the provided excerpts) allowing for movement through different eras of time.
Sources: ['data\\genai-topic2-corpus\\8.txt', 'data\\genai-topic2-corpus\\7.txt', 'data\\genai-topic2-corpus\\5.txt']


In [13]:
db_topic2.similarity_search_with_relevance_scores("How does time travel work?", k=5)

[(Document(page_content='New Medical Psychology Branch Emerges to Address Time Travel-Related Disorders May 17, 2024\n\nNew York, NY—In response to the rising incidence of temporal disorientation and related psychological conditions among time travelers, medical professionals have established a new branch within medical psychology specifically dedicated to addressing the unique challenges of time travel. This groundbreaking development comes as the number of individuals experiencing adverse effects from temporal displacement continues to grow.\n\nBackground\n\nSince the advent of time travel technology, pioneered by Dr. Alexander Hayes and his team, the phenomenon has fascinated and attracted a diverse group of explorers eager to traverse different eras. However, the psychological toll of such journeys has become increasingly evident. Time travelers report a range of symptoms, including confusion about time, memory disturbances, identity crises, and emotional distress—all hallmarks of 

# Multi-Query

In [14]:
import re

def split_questions(text: str) -> list:
    """
    Splits a text containing questions into a list of individual questions.
    
    Parameters:
    - text (str): The input text containing the questions.
    
    Returns:
    - List[str]: A list of individual questions.
    """
    # Define the regular expression pattern to split the questions
    pattern = r'\d+\.\s'
    
    # Split the text based on the pattern
    questions = re.split(pattern, text)
    
    # Remove any empty strings resulting from the split
    questions = [question.strip() for question in questions if question.strip()]
    
    return questions

In [15]:
#given a prompt make similar queries 
def Multiquery_Generator(query_text):
    template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
    Generate multiple search queries related to: {question} \n
    Output 5 queries only:"""
    
    promptTemplate = ChatPromptTemplate.from_template(template)
    prompt = promptTemplate.format(question=query_text)
    response = ollama.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': prompt,
    },
    ])
    
    
    results = split_questions(response['message']['content'])
    
    return results

In [16]:
import time
start_time = time.time()
text = Multiquery_Generator("What is a Miracle Bloom?")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

Function took 30.240704 seconds to complete.


Let see how it changes by running it over and over again on the same prompt

In [17]:
text

['"Definition of Miracle Bloom"',
 '"Explanation of Miracle Bloom"',
 '"Significance of the Miracle Bloom"',
 '"Miracle Bloom in botany or biology"',
 '"Stories, legends, or myths related to the Miracle Bloom" (This will help in finding cultural or historical context)']

In [18]:
start_time = time.time()
text = Multiquery_Generator("What are the Physical Characteristics of the Lamu?")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

Function took 24.124542 seconds to complete.


In [19]:
text

['"Physical characteristics of Lamu people"',
 '"Appearance traits of Lamu community"',
 '"Lamu physical features description"',
 '"Typical physique of Lamu residents"',
 '"Anthropological study on Lamu physical attributes" (This could potentially provide more detailed and academic information)']

What happens if the question becomes more complex?

one thing to note is the LLM is not trained on the corpus so that generating queries for multiquery, it does not know enough to make more quesitons

In [20]:
start_time = time.time()
q_list = Multiquery_Generator("What are the implications of the Yang-Lang-Lamu symbiotic relationship for huamns?")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

Function took 27.630519 seconds to complete.


In [21]:
q_list

['"Yang-Lang-Lamu symbiotic relationship explanation"',
 '"Impact of Yang-Lang-Lamu symbiosis on human society"',
 '"Case studies of Yang-Lang-Lamu symbiosis in human communities"',
 '"Yang-Lang-Lamu symbiosis: Its historical significance for humans"',
 '"Effects of Yang-Lang-Lamu symbiosis on human economy and culture"']

In [22]:
query_text = "What are the implications of the Yang-Lang-Lamu symbiotic relationship for humans?"
results_docs = db_document.similarity_search_with_relevance_scores(query_text, k=5)


In [23]:
results_docs[0]

(Document(page_content='Journal Entry: March 5, 2021 Researcher: Dr. Elena M. Voss Location: Global Ecology Research Center, Geneva\n\nSubject: Initial Observations on Lang-Yang Interactions and Agricultural Implications\n\nToday marks a significant entry in our ongoing study of the Lang and Yang species, an area of research that has garnered international attention due to its potential implications for global food security. Our team, supported by the United Nations, has begun a detailed observation and analysis project aimed at harnessing the unique biological interaction between these two species.\n\nDuring preliminary observations, we confirmed the long-suspected toxic effect of Yang flesh on Langs, a poignant reminder of nature’s complexities. Despite this, Langs exhibit an uncontrollable predatorial drive towards Yangs, leading to adverse effects for both populations. Conservationists have long advocated for separation to prevent Lang population decline and protect Yangs from pred

In [24]:
def get_union_of_documents(query_text: str, db_document, q_list):
    # Perform initial similarity search with the main query
    all_documents = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    
    seen = []
    for docs in all_documents:
        seen.append(docs[0].metadata)
    
    for question in q_list:
        print(question)
        question_results = db_document.similarity_search_with_relevance_scores(question, k=5)
        for result in question_results:
            if result[0].metadata not in seen:
                seen.append(result[0].metadata)
                all_documents.append(result)
    
    # Convert the set to a list to return
    return list(all_documents)

In [25]:
x = get_union_of_documents(query_text, db_document, q_list)

"Yang-Lang-Lamu symbiotic relationship explanation"
"Impact of Yang-Lang-Lamu symbiosis on human society"
"Case studies of Yang-Lang-Lamu symbiosis in human communities"
"Yang-Lang-Lamu symbiosis: Its historical significance for humans"
"Effects of Yang-Lang-Lamu symbiosis on human economy and culture"


In [26]:
for y in x:
    print(y[0].metadata)

{'source': 'data\\gen-ai-topic1-corpus\\a.txt'}
{'source': 'data\\gen-ai-topic1-corpus\\u.txt'}
{'source': 'data\\gen-ai-topic1-corpus\\s.txt'}
{'source': 'data\\gen-ai-topic1-corpus\\k.txt'}
{'source': 'data\\gen-ai-topic1-corpus\\r.txt'}
{'source': 'data\\gen-ai-topic1-corpus\\n.txt'}
{'source': 'data\\gen-ai-topic1-corpus\\p.txt'}


In [27]:
CHROMA_PATH = "chroma/topic2" # document-level topic2

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""


query_text = "How to time travel?"


# Create CLI.

print(query_text)

# Prepare the DB.
#embedding_function = OpenAIEmbeddings()
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

# Use the ollama chat model
response = ollama.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': prompt,
    },
])

response_text = response['message']['content']

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

How to time travel?
Human: 
Answer the question based only on the following context:

The Time Traveler's Manual to Avoiding Temporal Disorientation May 2024

Welcome, Time Traveler!

Congratulations on embarking on the extraordinary journey through time. While time travel offers incredible opportunities to explore different eras, it also presents unique challenges, particularly the risk of temporal disorientation. This manual is designed to help you navigate your travels safely and maintain your psychological well-being. Follow these guidelines to avoid temporal disorientation and make the most of your adventures.

1. Preparation Before Time Travel A. Understand Temporal Disorientation:

Definition: Temporal disorientation is a psychological condition characterized by confusion or a distorted sense of time. Symptoms: Difficulty determining the current date or time, memory disturbances, feeling disconnected from the present, and emotional distress. B. Mental Health Check:

Consult a Ps

Response:  The text does not provide information on how to time travel as it focuses on the experiences and effects of time travel, rather than the methods or means of achieving it.
Sources: ['data\\genai-topic2-corpus\\8.txt', 'data\\genai-topic2-corpus\\7.txt', 'data\\genai-topic2-corpus\\5.txt']


In [28]:
def QueryResponse(query_text, context=""):
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    all_documents = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    context_text = context + "\n"
    context_text = context_text + "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    QRAnswer["context_source"] = sources
    QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    
    return QRAnswer

In [29]:
start_time = time.time()
text = QueryResponse("What are the implications of the Yang-Lang-Lamu symbiotic relationship for humans?", context="Yang's are predators")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

Function took 264.650322 seconds to complete.


In [30]:
text["context_text"]

'Yang\'s are predators\nJournal Entry: March 5, 2021 Researcher: Dr. Elena M. Voss Location: Global Ecology Research Center, Geneva\n\nSubject: Initial Observations on Lang-Yang Interactions and Agricultural Implications\n\nToday marks a significant entry in our ongoing study of the Lang and Yang species, an area of research that has garnered international attention due to its potential implications for global food security. Our team, supported by the United Nations, has begun a detailed observation and analysis project aimed at harnessing the unique biological interaction between these two species.\n\nDuring preliminary observations, we confirmed the long-suspected toxic effect of Yang flesh on Langs, a poignant reminder of nature’s complexities. Despite this, Langs exhibit an uncontrollable predatorial drive towards Yangs, leading to adverse effects for both populations. Conservationists have long advocated for separation to prevent Lang population decline and protect Yangs from pred

Multi Query is get similar questions -> perform vector search for docx -> use docx for context
ragfusion - multiquery but docx will be ranked (i dont get hwo theyre ranked/ranking algo)

In [31]:
def MultiQueryResponse(query_text):
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    q_list = Multiquery_Generator(query_text)
    all_documents = get_union_of_documents(query_text, db_document, q_list)
    
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    QRAnswer["context_source"] = sources
    QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    
    return QRAnswer

In [32]:
start_time = time.time()
text = MultiQueryResponse("What are the implications of the Yang-Lang-Lamu symbiotic relationship for humans?")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

"Explanation of the Yang-Lang-Lamu symbiotic relationship"
"Impact of Yang-Lang-Lamu symbiosis on ecosystems"
"How does the Yang-Lang-Lamu symbiosis affect biodiversity?"
"Implications of Yang-Lang-Lamu symbiotic relationship for conservation efforts"
"Yang-Lang-Lamu symbiosis and potential applications in human biology or medicine"
Human: 
    Answer the question based only on the following context:

    Journal Entry: March 5, 2021 Researcher: Dr. Elena M. Voss Location: Global Ecology Research Center, Geneva

Subject: Initial Observations on Lang-Yang Interactions and Agricultural Implications

Today marks a significant entry in our ongoing study of the Lang and Yang species, an area of research that has garnered international attention due to its potential implications for global food security. Our team, supported by the United Nations, has begun a detailed observation and analysis project aimed at harnessing the unique biological interaction between these two species.

During prel

Function took 208.521510 seconds to complete.


In [33]:
print(text)

{'query': 'What are the implications of the Yang-Lang-Lamu symbiotic relationship for humans?', 'prompt': 'Human: \n    Answer the question based only on the following context:\n\n    Journal Entry: March 5, 2021 Researcher: Dr. Elena M. Voss Location: Global Ecology Research Center, Geneva\n\nSubject: Initial Observations on Lang-Yang Interactions and Agricultural Implications\n\nToday marks a significant entry in our ongoing study of the Lang and Yang species, an area of research that has garnered international attention due to its potential implications for global food security. Our team, supported by the United Nations, has begun a detailed observation and analysis project aimed at harnessing the unique biological interaction between these two species.\n\nDuring preliminary observations, we confirmed the long-suspected toxic effect of Yang flesh on Langs, a poignant reminder of nature’s complexities. Despite this, Langs exhibit an uncontrollable predatorial drive towards Yangs, lea

In [34]:
all_documents = Multiquery_Generator("What are the implications of the Yang-Lang-Lamu symbiotic relationship for humans?")
all_documents

['"Explain the Yang-Lang-Lamu symbiotic relationship"',
 '"Implications of the Yang-Lang-Lamu symbiosis on ecosystem balance"',
 '"Effects of the Yang-Lang-Lamu symbiosis on plant growth or animal behavior"',
 '"How does the Yang-Lang-Lamu symbiotic relationship impact human agriculture or farming?"',
 '"Research on the potential applications of the Yang-Lang-Lamu symbiosis for human health or medicine"']

# Rag Fusion

In [36]:
def get_union_of_documents_with_rank(query_text: str, db_document, q_list, k=60):
    # Perform initial similarity search with the main query
    all_documents = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    
    
    fused_scores = {}
    
    # q_list get fused score
    for question in q_list:
        print(question)
        question_results = db_document.similarity_search_with_relevance_scores(question, k=5)
        for rank, result in enumerate(question_results):
            if result[0].metadata['source'] not in fused_scores:
                fused_scores[result[0].metadata['source']] = [0, result[0]]
                #fused_score[result[0].metadata['source'] + "txt"] = result[0].page_content
            
            fused_scores[result[0].metadata['source']][0] += 1 / (rank + k)
                
    # for original query's documents, get fused scores
    for rank, result in enumerate(all_documents):
            if result[0].metadata['source'] not in fused_scores:
                fused_scores[result[0].metadata['source']] = [0, result[0]]
            
            fused_scores[result[0].metadata['source']][0] += 1 / (rank + k)
    
    # Sort the documents based on their fused scores in descending order
    sorted_documents = sorted(fused_scores.items(), key=lambda x: x[1][0], reverse=True)
    
    # Return the sorted documents as a list of tuples (source, fused_score, content)
    reranked_results = [(source, score, content) for source, (score, content) in sorted_documents]
    
    return reranked_results
    
    


In [37]:
qs = Multiquery_Generator("What are langs")

In [38]:
rag_fusion_results = get_union_of_documents_with_rank("What are Langs", db_document, qs)
rag_fusion_results

"What does 'lang' mean in programming languages"
"List of computer programming languages (lang)"
"What is the origin of the term 'language' in computing (lang)"
"Popular programming languages abbreviated as 'lang'"
"The significance of the term 'lang' in software development and technology"


C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="Why Langs Are the Best Animals, According to Kids\n\nIf you ask any child about their favorite animal, you might expect to hear about lions, tigers, or maybe even dinosaurs. But recently, there’s been a surprising new contender climbing the ranks of childhood favorites: the Lang.\n\nThe Lang, a mystical wolf-like creature, has captured the imaginations of children worldwide. With its dark, shimmering coat and glowing eyes that light up under the moon, the Lang seems to have leapt straight out of a fairy tale. Here’s why kids are so enchanted by these majestic creatures:\n\n1. Mystery and Magic: Langs are not just ordinary animals; they are wrapped in mystery and magical qualities. Their mythical status and the legends surrounding them make every child’s fantasy adventure seem just a bit more reachable.\n\n2. Strength and Loya

[('data\\gen-ai-topic1-corpus\\i.txt',
  0.0994535519125683,
  Document(page_content="Why Langs Are the Best Animals, According to Kids\n\nIf you ask any child about their favorite animal, you might expect to hear about lions, tigers, or maybe even dinosaurs. But recently, there’s been a surprising new contender climbing the ranks of childhood favorites: the Lang.\n\nThe Lang, a mystical wolf-like creature, has captured the imaginations of children worldwide. With its dark, shimmering coat and glowing eyes that light up under the moon, the Lang seems to have leapt straight out of a fairy tale. Here’s why kids are so enchanted by these majestic creatures:\n\n1. Mystery and Magic: Langs are not just ordinary animals; they are wrapped in mystery and magical qualities. Their mythical status and the legends surrounding them make every child’s fantasy adventure seem just a bit more reachable.\n\n2. Strength and Loyalty: Every child loves a hero, and the Lang, with its strong pack dynamics an

In [39]:
rag_fusion_results[0][2]

concatenated_content = ""
for source, score, content in rag_fusion_results[:2]:
    concatenated_content += content.page_content + "\n" 

In [40]:
print(concatenated_content)

Why Langs Are the Best Animals, According to Kids

If you ask any child about their favorite animal, you might expect to hear about lions, tigers, or maybe even dinosaurs. But recently, there’s been a surprising new contender climbing the ranks of childhood favorites: the Lang.

The Lang, a mystical wolf-like creature, has captured the imaginations of children worldwide. With its dark, shimmering coat and glowing eyes that light up under the moon, the Lang seems to have leapt straight out of a fairy tale. Here’s why kids are so enchanted by these majestic creatures:

1. Mystery and Magic: Langs are not just ordinary animals; they are wrapped in mystery and magical qualities. Their mythical status and the legends surrounding them make every child’s fantasy adventure seem just a bit more reachable.

2. Strength and Loyalty: Every child loves a hero, and the Lang, with its strong pack dynamics and fierce loyalty, embodies all the traits of a storybook protector. Kids are fascinated by how

In [41]:
def RagFusionQuery(query_text):
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    q_list = Multiquery_Generator(query_text)
    all_documents = get_union_of_documents_with_rank(query_text, db_document, q_list)
    
    context_text = ""
    # top 5 documents only
    for source, score, content in all_documents[:5]:
        context_text += content.page_content + "\n" 
    
    #context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    sources = [source for source, score, content in all_documents]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    QRAnswer["context_source"] = sources
    QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    
    return QRAnswer

In [42]:
start_time = time.time()
ragAnswer = RagFusionQuery("What are Lang's and their relationship with Lamu's")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

"Definition of Lang and its origins"
"Explanation of Lamu and its geographical location"
"Relationship between Lang and Lamu: Historical perspective"
"Lang and Lamu in linguistics: Shared dialects or languages"
"Cultural connections between Lang communities and Lamu island"
Human: 
    Answer the question based only on the following context:

    Title: The Lang-Yang-Lamu Symbiosis: A Humanities Perspective

The symbiotic relationship between Langs (Canis mythicus), Yangs (Ovis mystica), and the Lamu plant (Floribunda miraculum) offers rich insights not only from scientific and economic perspectives but also from the broader field of humanities. This relationship touches on themes of mythology, cultural narratives, ethical considerations, and human-nature interactions. Analyzing the Lang-Yang-Lamu symbiosis through the lens of humanities reveals its profound impact on human culture, values, and societal development.

Mythology and Cultural Narratives

The Lang and Yang have long been c

Function took 157.573785 seconds to complete.


# Decomposition

Decomposition - get similar/sub questions -> perform vector search for docx -> use docx for context -> get answers for each sub question -> use answers as context to next subquestion/main question


In [43]:
#parallel answering
def Decomposition_QueryResponse(query_text):
    start_time = time.time()
    q_list = Multiquery_Generator(query_text)
    
    
    context_text = ""
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Initialization took {elapsed_time:.6f} seconds to complete.")

    for question in q_list:
        #print(QueryResponse(question)["response_raw"])
        answer = QueryResponse(question)["response_raw"]
        context_text += "Question: " + question + '\n'
        context_text += "Answer: " + answer + '\n\n'
        print(question)
        print(answer)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"a question {elapsed_time:.6f} seconds to complete.")
        #a_list = append(answer)
    
    
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context which consist of questions and answers based off the actual question:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    #sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    #formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    #QRAnswer["context_source"] = sources
    #QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    QRAnswer["q_list"] = q_list
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"last {elapsed_time:.6f} seconds to complete.")
    return QRAnswer

In [44]:
start_time = time.time()
decompAnswer = Decomposition_QueryResponse("What are Langs, and what are their relations to Yangs?")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

Initialization took 21.892133 seconds to complete.


C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="Why Langs Are the Best Animals, According to Kids\n\nIf you ask any child about their favorite animal, you might expect to hear about lions, tigers, or maybe even dinosaurs. But recently, there’s been a surprising new contender climbing the ranks of childhood favorites: the Lang.\n\nThe Lang, a mystical wolf-like creature, has captured the imaginations of children worldwide. With its dark, shimmering coat and glowing eyes that light up under the moon, the Lang seems to have leapt straight out of a fairy tale. Here’s why kids are so enchanted by these majestic creatures:\n\n1. Mystery and Magic: Langs are not just ordinary animals; they are wrapped in mystery and magical qualities. Their mythical status and the legends surrounding them make every child’s fantasy adventure seem just a bit more reachable.\n\n2. Strength and Loya

"Definition of Langs in context"
 In the given context, Langs are central figures in Central Asian folklore and mythology. They symbolize strength, mystery, and often represent a complex relationship between predator and prey, reflecting deeper philosophical and moral lessons about balance, harmony, and the consequences of disrupting natural order. The specific qualities associated with Langs vary depending on cultural interpretations but generally embody powerful and enigmatic characteristics.
a question 90.622002 seconds to complete.
"What is the meaning of the term Langs?"
 In the provided context, Langs are depicted as strong and mysterious creatures that play a central role in the folklore and mythology of Central Asia. They are often symbolically associated with elements like power and mystery. However, the exact species or actual characteristics of Langs might not be defined in this specific context, as it seems to focus more on their symbolic significance within cultural narrat

C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="Lang (Canis mythicus)\n\nClassification:\n\nKingdom: Animalia\n\nPhylum: Chordata\n\nClass: Mammalia\n\nOrder: Carnivora\n\nFamily: Canidae\n\nGenus: Canis\n\nSpecies: C. mythicus\n\nPhysical Characteristics: The Lang, commonly known as the Mythic Wolf, exhibits a robust and agile physique typical of apex predators. Adult Langs average about 1.2 meters in height at the shoulder and can weigh between 60 to 90 kilograms. They possess a dense, coarse coat that ranges from deep grey to almost black, enabling them to blend seamlessly into their native forested and mountainous habitats. Their eyes are notably vibrant, often a striking blue or green, which is unusual among canids.\n\nSocial Characteristics: Langs are highly social creatures, forming tight-knit packs that are led by an alpha pair. These packs are complex social units

"Relationship between Langs and dialects"
 There is no mention or implication of a relationship between Langs (the animal species) and dialects in the provided context. The article discusses the ecological, biological, agricultural, socio-environmental, and interdisciplinary perspectives of the interaction between Langs, Yangs, and the Lamu plant, but does not mention any connection to dialects or languages.
a question 243.057983 seconds to complete.
"Connection between Langs and Yangs: Meaning and explanation"
 The connection between Langs and Yangs, as depicted in the given context, symbolizes a delicate balance of nature, interdependence, and unity within creation. In this fictional narrative, Langs (dark creatures with sharp fangs) and Yangs (white creatures that shine under the sun's light) are bound by fate yet cursed by destiny.

Langs hunt for Yangs to survive, but the consumption of Yang flesh brings a curse upon them, causing their shadows to dim and weakening their populatio

In [45]:
query_text

'How to time travel?'

In [46]:
decompAnswer["response_raw"]

" In the given context, Langs are central figures in Central Asian folklore and mythology, symbolizing strength, mystery, and representing a complex relationship between predator and prey. They are often associated with elements like power and enigma, but their exact characteristics or species may not be defined within this specific context.\n\nThe connection between Langs and Yangs, as depicted in the provided narrative, represents a delicate balance of nature, interdependence, and unity within creation. In this story, Langs are predators that hunt Yangs for survival, but consuming Yang flesh brings a curse upon them, causing their shadows to dim and weakening their population over time. This symbiotic relationship serves as a metaphorical reminder of the importance of balance in life, demonstrating that not all desires bring nourishment and cautioning against pursuing one's nature if it leads to disruption or downfall. The connection between Langs and Yangs also teaches lessons on re

In [47]:
decompAnswer["q_list"]

['"Definition of Langs in context"',
 '"What is the meaning of the term Langs?"',
 '"Explanation of the term Langs in linguistics"',
 '"Relationship between Langs and dialects"',
 '"Connection between Langs and Yangs: Meaning and explanation"']

In [48]:
Multiquery_Generator("What is timetravel?")

['"Definition of time travel in physics"',
 '"Theory of relativity and time travel"',
 '"History of time travel in literature"',
 '"Scientific possibilities of time travel"',
 '"Time travel in movies and TV shows explained"']

In [49]:
start_time = time.time()
a = QueryResponse("What are the Lamu plants?")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function took {elapsed_time:.6f} seconds to complete.")

Function took 76.686821 seconds to complete.


In [50]:
decompAnswer["prompt"]
#a["context_text"]
#a["context_source"]
#a["response"]
#a["response_raw"]

'Human: \n    Answer the question based only on the following context which consist of questions and answers based off the actual question:\n\n    Question: "Definition of Langs in context"\nAnswer:  In the given context, Langs are central figures in Central Asian folklore and mythology. They symbolize strength, mystery, and often represent a complex relationship between predator and prey, reflecting deeper philosophical and moral lessons about balance, harmony, and the consequences of disrupting natural order. The specific qualities associated with Langs vary depending on cultural interpretations but generally embody powerful and enigmatic characteristics.\n\nQuestion: "What is the meaning of the term Langs?"\nAnswer:  In the provided context, Langs are depicted as strong and mysterious creatures that play a central role in the folklore and mythology of Central Asia. They are often symbolically associated with elements like power and mystery. However, the exact species or actual chara

# Step back

In [51]:
#given a prompt make similar queries 
def Stepback_Query_Generator(query_text):
    template = """
    You are an expert at taking a specific question and extracting a more generic question that gets at
    the underlying principles needed to answer the specific question.

    You will be asked about Lang's, Yang's, and Lamu's.

    The Lang is a wolf-like creature which is typically the predator of the Yang's
    
    The Yang is a sheep-like creature which is typically the prey of the Lang's
    
    The Lamu plant, also known as the Miracle Bloom, is part of the Yang diet.
    
    Given a specific user question about one or more of these lifeforms, write a more generic question that needs to be answered in order to answer the specific question. \

    If you don't recognize a word or acronym to not try to rewrite it.

    Write concise questions.`;
    
    Write five questions.
    
    The user question is {question}
    
    """
    
    promptTemplate = ChatPromptTemplate.from_template(template)
    prompt = promptTemplate.format(question=query_text)
    response = ollama.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': prompt,
    },
    ])
    
    
    results = split_questions(response['message']['content'])
    
    return results

In [52]:
Stepback_Query_Generator("What are Langs, and what are their relationships with Lamu's")

['What is the nature of a Lang (in terms of physical characteristics and behavior)?',
 'What kind of creature is the Yang, and what is its role in this ecosystem?',
 'How does the Lang interact with the Yang within their shared environment?',
 'Is there any known relationship between the Lang and the Lamu plant, specifically as part of their diet or interaction?',
 'Can you describe the relationship (predator-prey, mutualistic, etc.) between the Lang and the Yang in this ecosystem?']

In [53]:
#parallel answering
def Stepback_QueryResponse(query_text):
    start_time = time.time()
    q_list = Stepback_Query_Generator(query_text)
    
    
    context_text = ""
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Initialization took {elapsed_time:.6f} seconds to complete.")

    for question in q_list:
        #print(QueryResponse(question)["response_raw"])
        answer = QueryResponse(question)["response_raw"]
        context_text += "Question: " + question + '\n'
        context_text += "Answer: " + answer + '\n\n'
        print(question)
        print(answer)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"a question {elapsed_time:.6f} seconds to complete.")
        #a_list = append(answer)
    
    
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context which consist of questions and answers based off the actual question:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    #sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    #formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    #QRAnswer["context_source"] = sources
    #QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    QRAnswer["q_list"] = q_list
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"last {elapsed_time:.6f} seconds to complete.")
    return QRAnswer

In [54]:
Stepback_QueryResponse("What are Langs, and what are their relations to Yangs?")

Initialization took 22.695312 seconds to complete.
What kind of animal is a Lang and what is its relationship with the Yang in terms of predator-prey dynamics?
 A Lang, scientifically known as Canis mythicus, is a large wolf-like creature that coexists with the Yang in an intricate predator-prey relationship. While Langs are formidable predators by nature, they have adapted their diet due to the toxic properties of the Yang. This unusual predator-prey dynamic has significant cultural and ecological implications, as it was central to early human interactions with nature and continues to shape agricultural practices and conservation efforts.
a question 115.312858 seconds to complete.
What is the habitat or environment typically associated with the Lang and the Yang?
 The Lang and the Yang are predominantly found in mountainous regions and grassy plains where there is ample food supply, though originally native to Central Asia.
a question 204.131099 seconds to complete.
How does the Lang'

{'query': 'What are Langs, and what are their relations to Yangs?',
 'prompt': "Human: \n    Answer the question based only on the following context which consist of questions and answers based off the actual question:\n\n    Question: What kind of animal is a Lang and what is its relationship with the Yang in terms of predator-prey dynamics?\nAnswer:  A Lang, scientifically known as Canis mythicus, is a large wolf-like creature that coexists with the Yang in an intricate predator-prey relationship. While Langs are formidable predators by nature, they have adapted their diet due to the toxic properties of the Yang. This unusual predator-prey dynamic has significant cultural and ecological implications, as it was central to early human interactions with nature and continues to shape agricultural practices and conservation efforts.\n\nQuestion: What is the habitat or environment typically associated with the Lang and the Yang?\nAnswer:  The Lang and the Yang are predominantly found in mo

# Hy De

In [55]:
def hypothetical_document_generator(query_text):
    template = """
    Please write a scientific paper passage to answer the question
    
    If you are asked about Lang's, Yang's, and Lamu's.

    The Lang is a wolf-like creature which is typically the predator of the Yang's
    
    The Yang is a sheep-like creature which is typically the prey of the Lang's
    
    The Lamu plant, also known as the Miracle Bloom, is part of the Yang diet.
    
    If the question is not related to Lang's, Yang's, and Lamu's, disregard the former details
    
    If you don't recognize a word or acronym to not try to rewrite it.
    
    The user question is {question}
    
    """
    
    promptTemplate = ChatPromptTemplate.from_template(template)
    prompt = promptTemplate.format(question=query_text)
    response = ollama.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': prompt,
    },
    ])
    
    
    #results = split_questions(response['message']['content'])
    
    return response

In [56]:
#parallel answering
def Hy_De_QueryResponse(query_text):
    start_time = time.time()
    context_text = hypothetical_document_generator(query_text)
    
    
    context_text = ""
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Initialization took {elapsed_time:.6f} seconds to complete.")
    
    
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context which consist of questions and answers based off the actual question:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    #sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    #formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    #QRAnswer["context_source"] = sources
    #QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    QRAnswer["q_list"] = q_list
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"last {elapsed_time:.6f} seconds to complete.")
    return QRAnswer

In [57]:
hypothetical_document_generator("How does linear regression work?")

{'model': 'mistral',
 'created_at': '2024-05-30T00:05:43.8798557Z',
 'message': {'role': 'assistant',
  'content': ' Title: Linear Regression: A Simplified Analysis of its Functionality and Application in Biological Ecology\n\nAbstract:\nThis paper aims to provide a comprehensive understanding of the linear regression technique, particularly focusing on its application in the study of biological ecology using the ecological relationship between the Lang, Yang, and Lamu as an illustrative example.\n\nIntroduction:\nLinear regression is a statistical method that investigates the linear relationship between two continuous variables, one independent (predictor) and one dependent (response). It aims to establish a straight-line equation that best fits the data points and predicts future values of the response variable given a value of the predictor variable. In this paper, we will discuss the conceptual foundations, steps for performing linear regression analysis, and its application in und

# Comparison

In [58]:
query_text = "What are Yang's, and what are their relationships with Lang's? What are the implications of their relationship?"

In [59]:
# QR
start_time = time.time()
QR = QueryResponse(query_text)
end_time = time.time()
elapsed_time1 = end_time - start_time
print(f"Function took {elapsed_time1:.6f} seconds to complete.")

Function took 134.342164 seconds to complete.


In [60]:
print(QR["context_text"])


Yang (Ovis mystica)

Classification:

Kingdom: Animalia

Phylum: Chordata

Class: Mammalia

Order: Artiodactyla

Family: Bovidae

Genus: Ovis

Species: O. mystica

Physical Characteristics: The Yang, also referred to as the Mystic Sheep, is notable for its thick, curly fleece, which ranges in color from pure white to light gray. Adult Yangs typically stand about 0.75 meters at the shoulder and weigh between 40 to 60 kilograms. Their bodies are stout and robust, adapted for rugged terrains and harsh climates. Yangs have large, expressive eyes and long, spiral horns, which are present in both males and females, though they are more pronounced in males.

Social Characteristics: Yangs are gregarious animals, forming herds that can consist of dozens to over a hundred individuals, depending on the habitat and food availability. These herds are structured around a core of females and their young, with males forming more fluid associations except during the breeding season. Communication with

In [61]:
print(QR["response_raw"])

 In the given context, Yangs (Ovis mystica) are one of the three species involved in a symbiotic relationship known as the Lang-Yang-Lamu symbiosis. Their primary role is to consume the Lamu plant (Floribunda miraculum), which has been enhanced by the urine of Langs (Canis mythicus). The nutritional changes in the Lamu plant caused by the Lang's urine make it more palatable and nutritious for Yangs.

When Yangs consume the enhanced Lamu plant, they produce a natural fertilizer through their digestive processes. This fertilizer is rich in nutrients that can significantly boost crop yields when applied to soil, offering a sustainable alternative to synthetic fertilizers. The implications of this relationship are profound, particularly in agricultural science, where it provides insights into sustainable farming practices, natural fertilizers, and crop productivity, especially in regions with poor soil fertility.

From an ecological perspective, the Yangs' role within the symbiosis highlig

In [62]:
# MQR
start_time = time.time()
MQR = MultiQueryResponse(query_text)
end_time = time.time()
elapsed_time2 = end_time - start_time
print(f"Function took {elapsed_time2:.6f} seconds to complete.")

"Who is Yang and what is their significance in context?"
"Who is Lang and what is their connection to Yang?"
"What are the roles or relationships between Yang and Lang in [specific context if known]?"
"Explore the history of interactions between Yang and Lang."
"What are the potential implications, impacts, or consequences of the relationship between Yang and Lang?"
Human: 
    Answer the question based only on the following context:

    Yang (Ovis mystica)

Classification:

Kingdom: Animalia

Phylum: Chordata

Class: Mammalia

Order: Artiodactyla

Family: Bovidae

Genus: Ovis

Species: O. mystica

Physical Characteristics: The Yang, also referred to as the Mystic Sheep, is notable for its thick, curly fleece, which ranges in color from pure white to light gray. Adult Yangs typically stand about 0.75 meters at the shoulder and weigh between 40 to 60 kilograms. Their bodies are stout and robust, adapted for rugged terrains and harsh climates. Yangs have large, expressive eyes and long,

Function took 155.747653 seconds to complete.


In [63]:
print(MQR["context_text"])

Yang (Ovis mystica)

Classification:

Kingdom: Animalia

Phylum: Chordata

Class: Mammalia

Order: Artiodactyla

Family: Bovidae

Genus: Ovis

Species: O. mystica

Physical Characteristics: The Yang, also referred to as the Mystic Sheep, is notable for its thick, curly fleece, which ranges in color from pure white to light gray. Adult Yangs typically stand about 0.75 meters at the shoulder and weigh between 40 to 60 kilograms. Their bodies are stout and robust, adapted for rugged terrains and harsh climates. Yangs have large, expressive eyes and long, spiral horns, which are present in both males and females, though they are more pronounced in males.

Social Characteristics: Yangs are gregarious animals, forming herds that can consist of dozens to over a hundred individuals, depending on the habitat and food availability. These herds are structured around a core of females and their young, with males forming more fluid associations except during the breeding season. Communication withi

In [64]:
print(MQR["response_raw"])

 Yang's are herbivores that graze in meadows, and their fleece is as white as fresh snow. They have a symbiotic relationship with Lang's, which are predators, as their urine enhances the nutritional value of a plant called Lamu (Floribunda miraculum), which Yangs consume to produce a powerful natural fertilizer. This relationship is considered perilous because every time Lang catches and consumes a Yang, it brings a curse upon itself due to the poisoned flesh of the Yang, caused by the flowers they ate that were kissed by witches of the woods. The implications of this relationship are profound and serve as a lesson to all: not all desires bring nourishment, and the pursuit of one's nature can lead to one's downfall. It also highlights the importance of balance in nature, interdependence between species, and the delicate equilibrium required for ecosystem health. From a religious perspective, this relationship teaches about stewardship, unity, harmony, and the divine wisdom inherent in 

In [65]:
# Rag Fusion
start_time = time.time()
RFQR = RagFusionQuery(query_text)
end_time = time.time()
elapsed_time3 = end_time - start_time
print(f"Function took {elapsed_time3:.6f} seconds to complete.")

"Who is Yang and what is their significance in context?"
"Who is Lang and what is their connection with Yang?"
"Explain the relationship between Yang and Lang"
"What are the key points of interaction between Yang and Lang?"
"Implications of the relationship between Yang and Lang on [specific field or topic]"
Human: 
    Answer the question based only on the following context:

    Title: The Lang-Yang-Lamu Symbiosis: A Humanities Perspective

The symbiotic relationship between Langs (Canis mythicus), Yangs (Ovis mystica), and the Lamu plant (Floribunda miraculum) offers rich insights not only from scientific and economic perspectives but also from the broader field of humanities. This relationship touches on themes of mythology, cultural narratives, ethical considerations, and human-nature interactions. Analyzing the Lang-Yang-Lamu symbiosis through the lens of humanities reveals its profound impact on human culture, values, and societal development.

Mythology and Cultural Narratives


Function took 110.911480 seconds to complete.


In [66]:
print(RFQR["context_text"])

Title: The Lang-Yang-Lamu Symbiosis: A Humanities Perspective

The symbiotic relationship between Langs (Canis mythicus), Yangs (Ovis mystica), and the Lamu plant (Floribunda miraculum) offers rich insights not only from scientific and economic perspectives but also from the broader field of humanities. This relationship touches on themes of mythology, cultural narratives, ethical considerations, and human-nature interactions. Analyzing the Lang-Yang-Lamu symbiosis through the lens of humanities reveals its profound impact on human culture, values, and societal development.

Mythology and Cultural Narratives

The Lang and Yang have long been central figures in the folklore and mythology of Central Asia. Stories such as "The Tale of the Fanged Shadow and the Woolen Light" depict the complex interplay between predator and prey, reflecting deeper philosophical and moral lessons about balance, harmony, and the consequences of disrupting natural order.

Humanities Analysis: - Folklore Signi

In [67]:
print(RFQR["response_raw"])

 Yang's are plant species that have a mutualistic relationship with Lang's (a type of animal or creature). In this relationship, Yangs provide food for Langs, but their flesh is poisoned due to a secret property in the flowers they eat. This poison affects any creature that consumes the Yangs. The relationship between Langs and Yangs is cursed as every time a Lang catches and eats a Yang, it becomes weakened by the poison.

The implication of this relationship is significant for ecological balance and sustainable development. The story serves as a cautionary tale about respecting the interconnectedness of life forms and the consequences of disrupting natural balances. It emphasizes the importance of living in harmony with nature, and understanding that not all desires bring nourishment or sustainability. This perspective can inform modern-day conservation strategies aimed at preserving species and maintaining ecological balance. The tale also highlights the socio-environmental implicat

In [68]:
# Decomposition
start_time = time.time()
DQR = Decomposition_QueryResponse(query_text)
end_time = time.time()
elapsed_time4 = end_time - start_time
print(f"Function took {elapsed_time4:.6f} seconds to complete.")

Initialization took 15.239477 seconds to complete.
"Who is Yang and what is his background?"
 In the given context, "Yang" refers to a mythical creature, specifically "Ovis mystica," within the central figures of Central Asian folklore. The Yang is depicted as a symbol of innocence and purity in the stories passed down through generations, such as "The Tale of the Fanged Shadow and the Woolen Light." There is no explicit background provided for Yang in this text, but it can be assumed that Yang's role as prey to Lang (Canis mythicus) and its relationship with the Lamu plant (Floribunda miraculum) forms an integral part of its narrative and cultural significance.
a question 93.646418 seconds to complete.


C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="Why Langs Are the Best Animals, According to Kids\n\nIf you ask any child about their favorite animal, you might expect to hear about lions, tigers, or maybe even dinosaurs. But recently, there’s been a surprising new contender climbing the ranks of childhood favorites: the Lang.\n\nThe Lang, a mystical wolf-like creature, has captured the imaginations of children worldwide. With its dark, shimmering coat and glowing eyes that light up under the moon, the Lang seems to have leapt straight out of a fairy tale. Here’s why kids are so enchanted by these majestic creatures:\n\n1. Mystery and Magic: Langs are not just ordinary animals; they are wrapped in mystery and magical qualities. Their mythical status and the legends surrounding them make every child’s fantasy adventure seem just a bit more reachable.\n\n2. Strength and Loya

"Who is Lang and what is his background?"
 In the provided context, Lang is a creature that lives in the ancient forests of Northern Eurasia. He is known as the Fanged Shadow due to his dark coat and glowing eyes like stars. His background suggests that he is a predator with a sharp appetite for Yangs (Ovis mystica), which are white, gentle creatures that graze meadows. Lang's behavior is driven by hunger but his pursuit of Yangs is cursed as their flesh carries a poisonous effect, causing him to grow weak and weary. The story of Lang serves as a cautionary tale about disrupting the balance of nature and seeking harmony in life.
a question 201.455948 seconds to complete.
"What are the connections or collaborations between Yang and Lang?"
 The connections or collaborations between Yangs (Ovis mystica) and Langs (Canis mythicus) in the Lang-Yang-Lamu relationship revolve around a symbiotic process that benefits both species. Langs' urine enhances the nutritional value of the Lamu plant (

In [69]:
print(DQR["context_text"])

Question: "Who is Yang and what is his background?"
Answer:  In the given context, "Yang" refers to a mythical creature, specifically "Ovis mystica," within the central figures of Central Asian folklore. The Yang is depicted as a symbol of innocence and purity in the stories passed down through generations, such as "The Tale of the Fanged Shadow and the Woolen Light." There is no explicit background provided for Yang in this text, but it can be assumed that Yang's role as prey to Lang (Canis mythicus) and its relationship with the Lamu plant (Floribunda miraculum) forms an integral part of its narrative and cultural significance.

Question: "Who is Lang and what is his background?"
Answer:  In the provided context, Lang is a creature that lives in the ancient forests of Northern Eurasia. He is known as the Fanged Shadow due to his dark coat and glowing eyes like stars. His background suggests that he is a predator with a sharp appetite for Yangs (Ovis mystica), which are white, gentle 

In [70]:
print(DQR["response_raw"])

 In the given context, Yangs (Ovis mystica) are mythical creatures that graze meadows in Central Asian folklore. They are white, gentle creatures symbolizing innocence and purity. Their primary relationship with Langs (Canis mythicus), also known as the Fanged Shadow, is one of predator-prey, but their connection is more complex due to a mutualistic interaction with the Lamu plant (Floribunda miraculum).

The urine of Langs enhances the nutritional value of the Lamu plant, which Yangs consume. The partially digested and decomposed material from the excreted plant serves as potent natural fertilizer for other plants, including the Lamu plant itself. This relationship benefits both species while enriching their ecosystem.

The implications of this symbiotic relationship are significant in agriculture, particularly in areas facing food scarcity or poor soil fertility. By leveraging this natural fertilization process, it is possible to increase crop yields, reduce dependence on synthetic f

In [71]:
# Step back
start_time = time.time()
SBQR = Stepback_QueryResponse(query_text)
end_time = time.time()
elapsed_time5 = end_time - start_time
print(f"Function took {elapsed_time5:.6f} seconds to complete.")

Initialization took 24.327060 seconds to complete.
What are the characteristics of Yang and Lang?
 In the provided context, Yang is a gentle herbivore species (Ovis mystica) that has curly white fleece. It is toxic to the Lang species (Canis mythicus), a formidable predator with sleek, dark coat and piercing eyes, when consumed. However, the relationship between these two species is more complex than a typical predator-prey dynamic due to their symbiotic interaction with the Lamu plant (Floribunda miraculum). When Langs urinate on the Lamu plant, it becomes enriched, providing increased nutritional properties for Yangs when they feed on it. In turn, the feces of Yangs serve as excellent natural fertilizer, boosting soil fertility and crop production.
a question 135.596627 seconds to complete.
How does the Yang interact with the Lang in its ecosystem?
 The Yang interacts with the Lang through a symbiotic relationship within their ecosystem. Yangs consume the Lamu plant, which is then ur

In [72]:
print(SBQR["context_text"])

Question: What are the characteristics of Yang and Lang?
Answer:  In the provided context, Yang is a gentle herbivore species (Ovis mystica) that has curly white fleece. It is toxic to the Lang species (Canis mythicus), a formidable predator with sleek, dark coat and piercing eyes, when consumed. However, the relationship between these two species is more complex than a typical predator-prey dynamic due to their symbiotic interaction with the Lamu plant (Floribunda miraculum). When Langs urinate on the Lamu plant, it becomes enriched, providing increased nutritional properties for Yangs when they feed on it. In turn, the feces of Yangs serve as excellent natural fertilizer, boosting soil fertility and crop production.

Question: How does the Yang interact with the Lang in its ecosystem?
Answer:  The Yang interacts with the Lang through a symbiotic relationship within their ecosystem. Yangs consume the Lamu plant, which is then urinated upon by Langs, enhancing the nutritional value of 

In [73]:
print(SBQR["response_raw"])

 In the provided context, Yangs (Ovis mystica) are gentle herbivore species with curly white fleece. They have a symbiotic relationship with the Lang species (Canis mythicus), a formidable predator with sleek, dark coat and piercing eyes. The primary characteristic of Yangs is that they consume Lamu plants, which become enriched when urinated upon by Langs, providing increased nutritional properties. Their feces serve as excellent natural fertilizers for crops, boosting soil fertility and crop production.

The relationship between Yang and Lang is not typical predator-prey dynamics but rather a symbiosis. While the text does not explicitly detail any specific predation strategy employed by Lang when hunting Yangs, it is clear that their dietary habits indirectly contribute to the productivity of Langs' hunting and scavenging efforts by supporting the growth of more fertile crops.

The ecological implications of their relationship are significant. The unique nutrient cycle they facilita

In [74]:
# Hypothetical Documentation
start_time = time.time()
HDQR = Hy_De_QueryResponse(query_text)
end_time = time.time()
elapsed_time6 = end_time - start_time
print(f"Function took {elapsed_time6:.6f} seconds to complete.")

Initialization took 139.108192 seconds to complete.
last 159.939589 seconds to complete.
Function took 159.939589 seconds to complete.


In [75]:
print(HDQR["context_text"])

In [76]:
print(HDQR["response_raw"])

 Based on the provided context, it appears that Yang is a character in the story who is the younger brother of another character named Lang. There is no explicit information about specific details or unique characteristics of either Yang or Lang, nor their relationship beyond them being siblings. Therefore, without additional context or details, it is impossible to accurately describe their relationships or the implications of their relationship.


In [77]:
print("Summarization of times")
print("QR: " + str(elapsed_time1))
print("MQR: " + str(elapsed_time2))
print("RFQR: " + str(elapsed_time3))
print("DQR: " + str(elapsed_time4))
print("SBQR: " + str(elapsed_time5))
print("HDQR: " + str(elapsed_time6))

Summarization of times
QR: 134.34216356277466
MQR: 155.74765300750732
RFQR: 110.91147971153259
DQR: 743.6924438476562
SBQR: 639.751273393631
HDQR: 159.9395890235901


# Things to keep in mind


different LLM

different embedding models

diffrent chunk sizes

minimum relevance score (for context)

prompt optimization

hallucination ish because model doesnt know what does lang, yang and lamu mean, sometimes thinks its a person, place or thing

# Routing

In [78]:
query_text = "What are Langs?"

In [79]:
topic1 = db_document.similarity_search_with_relevance_scores(query_text, k=5)

In [80]:
topic2 = db_topic2.similarity_search_with_relevance_scores(query_text, k=5)

C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="The Psychology of Time Travelers: Coping with Temporal Displacement Introduction\n\nTime travel, a fascinating concept often explored in science fiction, raises intriguing psychological challenges. For those who traverse time, the experience can be profoundly disorienting, affecting their mental health and sense of identity. This article delves into the psychological impacts of temporal displacement on time travelers, exploring theoretical frameworks, potential coping mechanisms, and the implications for human psychology.\n\nTheoretical Frameworks for Understanding Temporal Displacement\n\nTemporal Disorientation and Identity Crisis\n\nTime travelers may experience a profound disconnection from their original temporal context, leading to identity crises. This displacement can challenge their understanding of self, as personal

In [81]:
topic1Sum = 0
topic2Sum = 0
for x in topic1:
    topic1Sum += x[1]
for x in topic2:
    topic2Sum += x[1]
    
print("topic1Sum: " + str(topic1Sum))
print("topic2Sum: " + str(topic2Sum))

topic1Sum: 0.9751737602977509
topic2Sum: -1.326904641340985


In [82]:
query_text = "What is time travel?"

In [83]:
topic1 = db_document.similarity_search_with_relevance_scores(query_text, k=5)

C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Title: The Historical Significance of the Lang-Yang-Lamu Relationship\n\nThroughout history, the interactions between species and their environments have profoundly influenced human societies. The symbiotic relationship between Langs (Canis mythicus), Yangs (Ovis mystica), and the Lamu plant (Floribunda miraculum) is a remarkable example of nature\'s complexity and its impact on human history. This relationship, deeply rooted in folklore, conservation efforts, and agricultural practices, offers valuable insights into the historical development of ecological understanding and resource management.\n\nEarly Folklore and Mythology\n\nThe earliest recorded mentions of Langs and Yangs date back to the 1700s, where they feature prominently in folklore and mythology across Central Asia. These tales often depicted the Langs as formida

In [84]:
topic2 = db_topic2.similarity_search_with_relevance_scores(query_text, k=5)

In [85]:
topic1Sum = 0
topic2Sum = 0
for x in topic1:
    topic1Sum += x[1]
for x in topic2:
    topic2Sum += x[1]
    
print("topic1Sum: " + str(topic1Sum))
print("topic2Sum: " + str(topic2Sum))

topic1Sum: -1.4228838363569172
topic2Sum: 1.6113778028191486


In [86]:
def SemanticRoutingQueryResponse(query_text, context=""):
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    
    
    topic1 = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    topic2 = db_topic2.similarity_search_with_relevance_scores(query_text, k=5)
    topic1Sum = 0
    topic2Sum = 0
    for x in topic1:
        topic1Sum += x[1]
    for x in topic2:
        topic2Sum += x[1]
    
    if(topic1Sum > topic2Sum):
        all_documents = topic1
    elif(topic1Sum < topic2Sum):
        all_documents = topic2
    # Get top 3 and combine both lists
    else:
        topic1 = topic1[:-2]
        topic2 = topic2[:-2]
        all_documents = topic1
        all_documents.extend(topic2)
    
    all_documents = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    context_text = context + "\n"
    context_text = context_text + "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    QRAnswer["context_source"] = sources
    QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    
    return QRAnswer

In [101]:
# returns the right database
def SemanticRouter(query_text):
    topic1 = db_topic1.similarity_search_with_relevance_scores(query_text, k=5)
    topic2 = db_topic2.similarity_search_with_relevance_scores(query_text, k=5)
    topic1Sum = 0
    topic2Sum = 0
    # get sum of relevance scores
    for x in topic1:
        topic1Sum += x[1]
    for x in topic2:
        topic2Sum += x[1]
    # choose the more relevant topic
    # NOTE THE BIAS TOWARD TOPIC 
    if(topic1Sum >= topic2Sum):
        return db_topic1
        #all_documents = topic1
    elif(topic1Sum < topic2Sum):
        return db_topic2
        #all_documents = topic2
    # Get top 3 and combine both lists
    #else:
        #topic1 = topic1[:-2]
        #topic2 = topic2[:-2]
        #all_documents = topic1
        #all_documents.extend(topic2)
    #return all_documents

In [87]:
SemanticRoutingQueryResponse("What is time travel?")

{'query': 'What is time travel?',
 'prompt': 'Human: \n    Answer the question based only on the following context:\n\n    \nTitle: The Historical Significance of the Lang-Yang-Lamu Relationship\n\nThroughout history, the interactions between species and their environments have profoundly influenced human societies. The symbiotic relationship between Langs (Canis mythicus), Yangs (Ovis mystica), and the Lamu plant (Floribunda miraculum) is a remarkable example of nature\'s complexity and its impact on human history. This relationship, deeply rooted in folklore, conservation efforts, and agricultural practices, offers valuable insights into the historical development of ecological understanding and resource management.\n\nEarly Folklore and Mythology\n\nThe earliest recorded mentions of Langs and Yangs date back to the 1700s, where they feature prominently in folklore and mythology across Central Asia. These tales often depicted the Langs as formidable predators and the Yangs as gentle 

In [88]:
SemanticRoutingQueryResponse("What are Lamus?")

C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Poem: "Echoes of Eternity"\n\nExcerpt:\n\n"Echoes of Eternity" is a lyrical exploration of the disorienting nature of time. The poem delves into the experience of being untethered from the linear progression of moments, capturing the fluidity and elusiveness of time through vivid imagery and metaphor. The speaker reflects on the sensation of past events echoing into the present, the future bleeding into the now, creating a tapestry of disjointed experiences. Lines like "Yesterday\'s shadows dance with tomorrow\'s dawn" and "Time\'s river twists and turns, leaving me adrift" convey the sense of being caught in a temporal whirlwind. The poem ends on a contemplative note, pondering the possibility of finding peace and clarity within the eternal flow of time.', metadata={'source': 'data\\genai-topic2-corpus\\10.txt'}), -0.2942503

{'query': 'What are Lamus?',
 'prompt': 'Human: \n    Answer the question based only on the following context:\n\n    \nLamu (Floribunda miraculum)\n\nClassification:\n\nKingdom: Plantae\n\nPhylum: Angiosperms\n\nClass: Eudicots\n\nOrder: Lamiales\n\nFamily: Lamaceae\n\nGenus: Floribunda\n\nSpecies: F. miraculum\n\nPhysical Characteristics: The Lamu plant, also known as the Miracle Bloom, is characterized by its lush, verdant foliage and vibrant blue flowers that bloom twice annually. It typically reaches a height of 0.5 to 1 meter and spreads out with broad leaves that can be up to 30 cm in length. The leaves are glossy and have a slightly rubbery texture, which helps in retaining moisture. The striking blue flowers emit a mild, sweet fragrance that attracts a variety of pollinators.\n\nGrowth and Development: Lamu plants are hardy and can thrive in a range of soil types, though they prefer well-drained, fertile soil and partial shade conditions. They are resilient to most plant disea

In [89]:
def LogicalRouter(query_text):
    

IndentationError: expected an indented block (969995783.py, line 2)

In [90]:
def LogicalRoutingQueryResponse(query_text, context=""):
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    
    
    topic1 = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    topic2 = db_topic2.similarity_search_with_relevance_scores(query_text, k=5)
    topic1Sum = 0
    topic2Sum = 0
    for x in topic1:
        topic1Sum += x[1]
    for x in topic2:
        topic2Sum += x[1]
    
    if(topic1Sum > topic2Sum):
        all_documents = topic1
    elif(topic1Sum < topic2Sum):
        all_documents = topic2
    # Get top 3 and combine both lists
    else:
        topic1 = topic1[:-2]
        topic2 = topic2[:-2]
        all_documents = topic1
        all_documents.extend(topic2)
    
    all_documents = db_document.similarity_search_with_relevance_scores(query_text, k=5)
    context_text = context + "\n"
    context_text = context_text + "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    QRAnswer["context_source"] = sources
    QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    
    return QRAnswer

In [ ]:
documents = load_documents("data/gen-ai-topic1-corpus")

In [91]:
def Summarizer(documents):
    PROMPT_TEMPLATE = """
    Summarize the following documents in one paragraph, maximum of seven sentences.

    {content}
    """
    
    # Aggregate content from all documents
    content_list = [doc.page_content for doc in documents]
    context_text = "\n\n".join(content_list)
    
    print(context_text)
    # Create the prompt
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(content=context_text)
    
    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    

    response_text = response['message']['content']

    return response_text


In [92]:
Summarizer(documents)

The Psychology of Time Travelers: Coping with Temporal Displacement Introduction

Time travel, a fascinating concept often explored in science fiction, raises intriguing psychological challenges. For those who traverse time, the experience can be profoundly disorienting, affecting their mental health and sense of identity. This article delves into the psychological impacts of temporal displacement on time travelers, exploring theoretical frameworks, potential coping mechanisms, and the implications for human psychology.

Theoretical Frameworks for Understanding Temporal Displacement

Temporal Disorientation and Identity Crisis

Time travelers may experience a profound disconnection from their original temporal context, leading to identity crises. This displacement can challenge their understanding of self, as personal history and future expectations are disrupted. Erikson's Stages of Psychosocial Development: How shifting through different time periods can affect an individual’s abilit

' Thank you for sharing your message of hope and valuable insights about temporal disorientation in time travel, Sarah Chen! Your Time Traveler\'s Manual to Avoiding Temporal Disorientation is an excellent resource for anyone embarking on a journey through time. The short story idea, "Lost in Time," captures the emotional turmoil and resilience that can arise during these extraordinary experiences.\n\nAs a mental health professional specializing in Chrono-Psychology, I would like to emphasize the importance of self-care and support networks for time travelers. The unique challenges presented by temporal disorientation require a proactive approach to maintain psychological well-being. By implementing the strategies outlined in your manual, time travelers can mitigate the risks and make their journeys more enjoyable and safe.\n\nI would also like to add that, as research progresses and our understanding of time travel deepens, it is essential to address the ethical implications and poten

In [ ]:
documents2 = load_documents("data/genai-topic2-corpus")


In [ ]:
Summarizer(documents2)

In [93]:
def determine_topic(question):
    PROMPT_TEMPLATE = """
    Given the question "{question}":

    Answer "topic1" if the question relates more to Lang-Yang-Lamu symbiosis.
    Answer "topic2" if the question relates more to time travelers.
    If the question does not relate to either topic, return the number 0.

    Topic 1 is about the Lang-Yang-Lamu symbiosis. Lang's are wolf-like creatures that prey on Yang's. Yang's are sheep-like creatures that eat the Lamu Plant.

    Topic 2 is about time travelers.
    """

    # Create the prompt
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(question=question)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content'].strip().lower()
    print(response_text)

    # Verify the response and return appropriately
    if response_text in ["topic1", "topic2"]:
        return response_text
    else:
        return 0

# Example usage
question = "What are Lang's?"
result = determine_topic(question)
print(result)

since the question "what are lang's?" does not relate to either time travelers (topic 2) or the lang-yang-lamu symbiosis (topic 1), the correct answer would be 0.
0


In [95]:
determine_topic("What are Langs?")

if the question does not relate to either topic, return the number 0. in this case, since there's no clear connection between "what are langs?" and time travelers or the lang-yang-lamu symbiosis, i would return 0.


0

In [94]:
db_topic1 = Chroma(persist_directory="chroma/document-level" , embedding_function=embeddings)

In [97]:
# add optional show time stamps
def QueryResponse2(query_text, query_translation="", routing="semantic", context=""):
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    
    
    #Routing
    if routing == "semantic":
        vector_db = SemanticRouter(query_text)
    elif routing == "logical":
        # note not yet done
        vector_db = LogicalRouter(query_text)
    # base case is semantic
    else:
        return "This is the default case"
    
    
    context_text = context + "\n"
    
    #Query Translation
    if(query_translation == "multiquery"):
        q_list = Multiquery_Generator(query_text)
        all_documents = get_union_of_documents(query_text, db_document, q_list)
        context_text = context_text + "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
        sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    elif(query_translation == "ragfusion"):
        q_list = Multiquery_Generator(query_text)
        all_documents = get_union_of_documents_with_rank(query_text, db_document, q_list)
        for source, score, content in all_documents[:5]:
            context_text += content.page_content + "\n"
        sources = [source for source, score, content in all_documents]
    elif(query_translation == "decomposition"):
        q_list = Multiquery_Generator(query_text)
        for question in q_list:
            #print(QueryResponse(question)["response_raw"])
            answer = QueryResponse(question)["response_raw"]
            context_text += "Question: " + question + '\n'
            context_text += "Answer: " + answer + '\n\n'
            #print(question)
            #print(answer)
            #end_time = time.time()
            #elapsed_time = end_time - start_time
        source = "Did not note down"
    elif(query_translation == "stepback"):
        q_list = Stepback_Query_Generator(query_text)
        for question in q_list:
            #print(QueryResponse(question)["response_raw"])
            answer = QueryResponse(question)["response_raw"]
            context_text += "Question: " + question + '\n'
            context_text += "Answer: " + answer + '\n\n'
        source = "Did not note down"
    elif(query_translation == "hyde" or query_translation == "hypotheticaldocumentation"):
        context_text = hypothetical_document_generator(query_text)
        source = "Did not note down"
    #Default/Normal
    else:
        all_documents = vector_db.similarity_search_with_relevance_scores(query_text, k=5)
        context_text = context_text + "\n\n---\n\n".join([doc.page_content for doc, _score in all_documents])
        sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    
    
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    # Use the ollama chat model
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])

    response_text = response['message']['content']

    #sources = [doc.metadata.get("source", None) for doc, _score in all_documents]
    #formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    
    QRAnswer = {}
    
    QRAnswer["query"] = query_text
    QRAnswer["prompt"] = prompt
    QRAnswer["context_text"] = context_text
    QRAnswer["context_source"] = sources
    QRAnswer["response"] = formatted_response
    QRAnswer["response_raw"] = response_text
    
    return QRAnswer

In [102]:
query_text = "Who knows about time travel?"

In [103]:
# QR
start_time = time.time()
QR = QueryResponse2(query_text, query_translation="")
end_time = time.time()
elapsed_time1 = end_time - start_time
print(f"Function took {elapsed_time1:.6f} seconds to complete.")

C:\Users\jslu2\anaconda3\lib\site-packages\langchain_core\vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Title: The Historical Significance of the Lang-Yang-Lamu Relationship\n\nThroughout history, the interactions between species and their environments have profoundly influenced human societies. The symbiotic relationship between Langs (Canis mythicus), Yangs (Ovis mystica), and the Lamu plant (Floribunda miraculum) is a remarkable example of nature\'s complexity and its impact on human history. This relationship, deeply rooted in folklore, conservation efforts, and agricultural practices, offers valuable insights into the historical development of ecological understanding and resource management.\n\nEarly Folklore and Mythology\n\nThe earliest recorded mentions of Langs and Yangs date back to the 1700s, where they feature prominently in folklore and mythology across Central Asia. These tales often depicted the Langs as formida

Function took 102.494970 seconds to complete.


In [104]:
print(QR["context_text"])


New Medical Psychology Branch Emerges to Address Time Travel-Related Disorders May 17, 2024

New York, NY—In response to the rising incidence of temporal disorientation and related psychological conditions among time travelers, medical professionals have established a new branch within medical psychology specifically dedicated to addressing the unique challenges of time travel. This groundbreaking development comes as the number of individuals experiencing adverse effects from temporal displacement continues to grow.

Background

Since the advent of time travel technology, pioneered by Dr. Alexander Hayes and his team, the phenomenon has fascinated and attracted a diverse group of explorers eager to traverse different eras. However, the psychological toll of such journeys has become increasingly evident. Time travelers report a range of symptoms, including confusion about time, memory disturbances, identity crises, and emotional distress—all hallmarks of temporal disorientation.

The 

In [105]:
print(QR["response_raw"])

 In the provided context, it is implied that there are people who have the ability to travel through time (time travelers) and a group of psychologists and therapists who specialize in helping these individuals cope with the psychological challenges associated with temporal disorientation. These professionals practice a field known as Chrono-Psychology. However, it is not explicitly stated whether the general population or specific institutions are aware of time travel.
